The Goal is to predict weapon type used based on the year, target type, country, and attack type

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import sklearn.model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Read files
df = pd.read_excel('files/GTD/globalterrorismdb_0522dist.xlsx')

In [13]:
df_interest = df[['iyear', 'country_txt', 'attacktype1', 'targtype1', 'weaptype1_txt', 'nkill', 'nwound', 'success', 'crit1', 'crit2', 'crit3']]
df_cleaned = df_interest[df_interest['weaptype1_txt'] != 'Unknown']
df_cleaned = df_cleaned.dropna().reset_index()
display(df_cleaned)

,index,iyear,country_txt,attacktype1,targtype1,weaptype1_txt,nkill,nwound,success,crit1,crit2,crit3
0,5,1970,United States,2,3,Firearms,0.0,0.0,1,1,1,1
1,6,1970,Uruguay,1,3,Firearms,0.0,0.0,0,1,1,1
2,7,1970,United States,3,21,Explosives,0.0,0.0,1,1,1,1
3,8,1970,United States,7,4,Incendiary,0.0,0.0,1,1,1,1
4,9,1970,United States,7,2,Incendiary,0.0,0.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
173340,209699,2020,Yemen,3,12,Explosives,0.0,0.0,1,1,1,1
173341,209700,2020,Nepal,7,2,Incendiary,0.0,0.0,1,1,1,1
173342,209703,2020,Germany,7,4,Incendiary,0.0,0.0,1,1,1,0
173343,209704,2020,Afghanistan,2,14,Firearms,1.0,0.0,1,1,1,1


In [28]:
# liss attempt, linear regression 
import pandas as pd
import statsmodels.api as sm

# Create dummy variables for all categorical variables 
country_dummies = pd.get_dummies(df['country_txt'], prefix='country', drop_first=True)
attack_type_dummies = pd.get_dummies(df['attacktype1_txt'], prefix='attack type', drop_first=True)
target_dummies = pd.get_dummies(df['targtype1_txt'],prefix='target type', drop_first=True)
crit1_dummies = pd.get_dummies(df['crit1'],prefix='crit1', drop_first=True)
crit2_dummies = pd.get_dummies(df['crit2'],prefix='crit2', drop_first=True)
crit3_dummies = pd.get_dummies(df['crit3'],prefix='crit3', drop_first=True)


# Concatenate the dummy variables with the DataFrame
new_df = pd.concat([df, country_dummies, attack_type_dummies, target_dummies, crit1_dummies, crit2_dummies, crit3_dummies], axis=1)

# Specify the columns to be included in X. Ensure all are numeric
X = new_df[['iyear', 'crit1'] + 
         attack_type_dummies.columns.tolist() + country_dummies.columns.tolist() + crit1_dummies.columns.tolist() + target_dummies.columns.tolist() + crit2_dummies.columns.tolist() + crit3_dummies.columns.tolist()].astype(float) 
y = new_df[['nkill']]

# Add a constant to the features to account for the intercept in the model
X_with_const = sm.add_constant(X)

# Fit the model to estimate the time for change regression 
model = sm.OLS(y, X_with_const).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  nkill   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 26 Feb 2024   Prob (F-statistic):                nan
Time:                        16:50:43   Log-Likelihood:                    nan
No. Observations:              209706   AIC:                               nan
Df Residuals:                  209469   BIC:                               nan
Df Model:                         236                                         
Covariance Type:            nonrobust                                         
                                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [ ]:
df_encoded = pd.get_dummies(df_cleaned, columns=['country_txt', 'attacktype1', 'targtype1'])

X = df_encoded.drop('weaptype1_txt', axis=1)
y = df_encoded['weaptype1_txt']
X_intercept = sm.add_constant(X)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_intercept, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=5).fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
'''
enc = OneHotEncoder()
# Transforming categorical values to matrix
y = enc.fit_transform(y.values.reshape(-1,1))

x_cat = x['country_txt']
x_cat = enc.fit_transform(x_cat.values.reshape(-1,1))
x_cat = x_cat.toarray()

x_num = x.drop(columns={'country_txt'})
x_num = x_num.values

X = np.concatenate([x_cat, x_num], axis = 1)
'''